export cells as modules?



In [ ]:
var fs = require('fs');
var path = require('path');
var rimraf = require('rimraf');
var importer = require('../Core');
var mkdirpSync = importer.import('mkdirp');
var getExtension = importer.import('cell extension')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var MODULE_OUTPUT = __dirname + '/.modules';
if(fs.existsSync(MODULE_OUTPUT)) {
    rimraf.sync(MODULE_OUTPUT);
}

function exportNotebook(notebook, project) {
    if(!project) {
        project = __dirname + '/..';
    }
    const notebooks = typeof notebook === 'string' ? [notebook] : notebook;
    return notebooks.reduce((cells, notebook) => {
        const name = path.basename(notebook).replace(/\.ipynb/ig, '');
        const parent = path.resolve(path.dirname(notebook)).replace(path.resolve(project), '');
        mkdirpSync(path.join(MODULE_OUTPUT, parent, name));

        return importer.getCells(notebook, ['*', 'code', 'markdown', 'raw'])
            .reduce((results, cell, i) => {
                const extension = getExtension(cell, notebook);
                // TODO: get previous markdown and extract name that leads back to the current cell
                const cellPath = path.join(MODULE_OUTPUT, parent, name, 'cell-' + i + extension);
                fs.writeFileSync(cellPath, cell.source.join(''));
                results.push(cellPath);
                return results;
            }, cells);
    }, []);
}
module.exports = exportNotebook;


get cell extension?


In [ ]:

function getExtension(cell, notebook) {
    var extension;
    if(cell.cell_type === 'markdown') {
        extension = '.md';
    } else if(cell.cell_type === 'raw') {
        extension = '.txt';
    } else if(cell.language === 'javascript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.js'
        } else {
            extension = '.js';
        }
    } else if(cell.language === 'powershell') {
        extension = '.ps1';
    } else if(cell.language === 'csharp') {
        extension = '.cs';
    } else if(cell.language === 'python') {
        extension = '.py';
    } else if(cell.language === 'typescript') {
        if((cell.source || [cell.code]).join('').match(/describe\s*\(/igm)) {
            extension = '.spec.ts'
        } else {
            extension = '.ts';
        }
    } else if(cell.language === 'bash') {
        extension = '.sh';
    } else {
        throw 'unknown language or cell type: ' + (cell.filename || notebook) + ' (' + cell.cell_type + ',' + cell.language + ')';
    }
    return extension;
}
module.exports = getExtension;


export all notebooks?

Export all notebooks to a structured folder where each cell has it's own file for linting?



In [ ]:
var glob = require('glob');
var path = require('path');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

function exportAll(project) {
    const notebooks = glob.sync('**/*.ipynb', {cwd: project});
    return exportNotebook(notebooks.map(n => path.join(project, n)), project);
}
module.exports = exportAll;


test notebook export?



In [ ]:
var path = require('path');
var glob = require('glob');
var assert = require('assert');
var fs = require('fs');
var importer = require('../Core');
var exportNotebook = importer.import('notebook.ipynb[export cells modules]');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = path.resolve(__dirname + '/../');
var MODULE_OUTPUT = PROJECT_PATH + '/Utilities/.modules';

var sortNumeric = (that) => {
    return that.sort((a, b) => parseInt(a.split(/[^0-9]/ig).join('')) - parseInt(b.split(/[^0-9]/ig).join('')));
}

describe('notebook export service', () => {
    it('should export this file', () => {
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        assert(path.basename(files[0]) === 'cell-0.md');
        assert(path.basename(files[1]) === 'cell-1.js');
        assert(path.basename(files[2]) === 'cell-2.md');
    })
    
    it('should have comparable cells', () => {
        var importCells = importer.interpret(['notebook.ipynb[0]', 'notebook.ipynb[1]', 'notebook.ipynb[2]']);
        exportNotebook(path.resolve(PROJECT_PATH + '/Utilities/notebook.ipynb'));
        var files = glob.sync('Utilities/notebook/*', {cwd: MODULE_OUTPUT});
        sortNumeric(files);
        var fsCell0 = fs.readFileSync(path.join(MODULE_OUTPUT, files[0])).toString();
        var fsCell1 = fs.readFileSync(path.join(MODULE_OUTPUT, files[1])).toString();
        var fsCell2 = fs.readFileSync(path.join(MODULE_OUTPUT, files[2])).toString();
        assert(fsCell0 === importCells[0].markdown.join(''));
        assert(fsCell1 === importCells[0].fresher, fsCell1.length + ' != ' + importCells[0].fresher.length)
        assert(fsCell2 === importCells[1].markdown.join(''))
    })
    
    it('should export all notebooks', () => {
    //    var exported = exportAll(PROJECT_PATH);
    //    assert(exported.length > 0);
    })
})


import all cell modules?

import files back in to cells? (two-way workflow)


In [ ]:
var fs = require('fs');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Documents/jupytangular2/Utilities/.modules';
var project = PROFILE_PATH + '/Documents/jupytangular2';

var cells = glob.sync('**/cell-*', {cwd: PROJECT_PATH});
for(const c of cells) {
    const cell = path.basename(c);
    const notebook = path.basename(path.dirname(c));
    const parent = path.basename(path.dirname(path.dirname(c)));
    if(parent === 'jupytangular2') {
        continue;
    }
    const nb = JSON.parse(fs.readFileSync(path.join(project, parent, notebook + '.ipynb')));
    let counter = 0;
    for(const i in nb.cells) {
        if(!nb.cells.hasOwnProperty(i)) {
            continue;
        }
        // TODO: reimport all cells
        if(nb.cells[i].cell_type === 'code') {
            if(cell === 'cell-' + counter + '.js') {
                nb.cells[i].source = fs.readFileSync(path.join(PROJECT_PATH, c)).toString().split('\n');
            }
            counter++;
        }
    }
    fs.writeFileSync(path.join(project, parent, notebook + '.ipynb'), JSON.stringify(nb, null, 2));
}


In [ ]:
// TODO: delete .modules if everything checks out

// TODO: import tests


TODO:

import module as notebook cell?

import gist as notebook

import random instructions as notebook

rules:
- testing cells have zero markdown before them,
- all public functions are made public using module.export or global namespace
- use describe( test blocks to describe parsing and replacement information for the functions it is testing, i.e. function utility(root) would have a describe block describe('utility(root)') neatly formated and parsed for calendar commands
- every file is one feature
- every cell or export should be a single purpose component with proper includes/dependencies
- every cell must be under 100 lines
- every function must be testable insolation
- every function should be made accesible from the command line using module.exports
- every function should be runnable from notebooks using typeof $$ !== 'undefined'


```javascript

if(typeof $$ !== 'undefined') {
    $$.async();
    exportAndDeploy('../Frameworks/zuora to eloqua.ipynb')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}

```
- every module should have a markdown title, at least one question that the code intends to answer (how to?), includes at least (where name is the name of any function in the code block):


```javascript

...
function <name> () {
...
}
...
exports = <name>
...

```

 - eliminate circular dependencies the same way unintended recursiion is avoided, create a condition:
 
ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
var funcA = require('ModuleA.js');
```

Becomes:

ModuleA.js
```
var funcB = require('ModuleB.js');
```

ModuleB.js
```
if(typeof funcA === 'undefined') {
    var funcA = require('ModuleA.js');
}
```

 - parameters are listed in most specific left to least specific right, i.e. function(filter, context) would mean filter is only used for this function, whereas context may be passed in to this function as well as other functions.  "filter" is on the left because it is specifically used just for this function, context is on the right because it might contain a path on the filesystem, or some options.
 - Entry cells, i.e. cells that are intended to be called by a service, coordinate interactions between multiple services - should not contain catch blocks so that the task scheduler will fail and log the last error.




get imports recursively?

TODO: use this in file-system graph

TODO: AST compatible map of import functions across all languages
Python `__import__`, Java: `import`, C#: `using`, C++: `#import`.  Basic functionality of JetBrains intellisense.



In [ ]:
var importer = require('../Core');

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

function matchImports(code) {
    var imports = regexToArray(/(importNotebook|import)\(((\s*([\['"\{])\s*([\s\S]*?)\s*\3[,\s]*)+)\)/ig, code, 2);
    return imports.map(i => {
        const importStrings = regexToArray(/([\['"\{])\s*([\s\S]*?)\s*\1/ig, i, 2);
        if(i.indexOf('[') > -1) {
            return importStrings;
        }
        return importStrings[0];
    });
}

function getImportsRecursively(imports, cell) {
    if(typeof imports[cell.id] !== 'undefined') {
        return imports[cell.id];
    }
    cell.code = cell.code.replace(/require\('\.\.\/Core'\).*\s*/ig, 'importer.import(\'import notebook.ipynb\');\n');
    imports[cell.id] = matchImports(cell.code);
    imports[cell.id].forEach(arr => {
        if(typeof arr === 'string') {
            arr = [arr];
        }
        arr.forEach(i => {
            const result = importer.interpret(i);
            if(result.length) {
                result.forEach(r => {
                    getImportsRecursively(imports, r);
                });
            } else {
                getImportsRecursively(imports, result);
            }
        });
    });
    return imports[cell.id];
}
module.exports = getImportsRecursively;



replace notebook imports?

automatic jupyter dependency injector?

TODO: Use and AST instead of regexp



In [ ]:
var _ = require('lodash');
var importer = require('../Core');

var REGEX_EXPORTS = /exports[^=]*?\s*=\s*([^\s;]*)/igm;
var REGEX_NAMED = /function\s+([^\( ]*)\s*\(/igm

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};

// How to escape a string for regex?
function escapeRegExp(str) {
    return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, '\\$&');
}

function getExports(code) {
    const named = regexToArray(REGEX_NAMED, code, 1);
    const namedExports = regexToArray(REGEX_EXPORTS, code, 1);
    return _.intersection(named, namedExports);
}

// TODO: get previous markdown and extract name that leads back to the current cell
function niceName(cell) {
    cell.questions.sort((a, b) => a.length - b.length);
    return cell.questions[0].replace('?', '').replace(/[^a-z0-9]+|\btest\b/igm, ' ').trim().replace(/\s+/igm, '-') + getExtension(cell);
}

function addImports(exportedCode, imports) {
    const newExports = getExports(exportedCode);
    // TODO: include all functional references from cells above this current cell
    Object.keys(imports).forEach(k => {
        const includedCell = imports[k];
        const funcName = getExports(includedCell.code)[0];
        if(funcName
            && newExports.indexOf(funcName) === -1
            && exportedCode.indexOf(funcName + '(') > -1
            && !exportedCode.match(new RegExp('(let|var)\\s+' + funcName + '|(let|var)\\s+\\{[^\\}]*?' + funcName, 'ig'))
            && includedCell.questions.length > 0) {
            // TODO: replace this with a polyfill across ALL notebooks
            exportedCode = 'var ' + funcName + ' = (() => { const r = require(\'./' + niceName(includedCell) + '\'); return typeof r === \'function\' ? r : (r && typeof r[Object.keys(r)[0]] === \'function\' ? r[Object.keys(r)[0]] : r); })();\n' + exportedCode;
        }
    });
    return exportedCode;
}

// TODO: exclude Core
function replaceObvious(code) {
    var result = code.replace(/.*typeof \$\$[\s\S]*?\}.*/igm, '');
    const importerReg = /importer\.(interpret|runAllPromises|runInNewContext|getCells|streamJson)/igm;
    const excludeCore = (true || !result.match(importerReg)) && !code.includes('tmpGlobal.importer')
    if(excludeCore !== true) {
        result = result.replace(/require\('\.\.\/Core'\)\s*;*\s*/ig, 'importer.import(\'import notebook.ipynb\');');
    } else {
        result = result.replace(/.*require\('\.\.\/Core'\)\s*;*\s*/ig, '').replace(importerReg, '$1')
    }
    return result
//        .replace(/\bvar\b/igm, 'let');
}

// TODO: use an AST for all of this static analysis
function replaceImports(exports, cell, allImports) {
    cell.name = niceName(cell);
    // TODO: use AST for this, doesn't work in quotes
    if(cell.filename.includes('injectImports()[1]')) {
        return cell.code;
    }
    var newCell = replaceObvious(cell.code);
    exports.forEach(i => {
        let result = typeof i === 'string' ? allImports[i] : i.map(i => allImports[i]);
        if (typeof result === 'undefined' || result.length === 0) {
            return;
        }
        if(result.length > 1) {
            newCell = newCell.replace(
                new RegExp('((importer\\.)*import(Notebook)*)\\([\\s\\S]*?' + escapeRegExp(typeof i === 'string' ? i : i[0]) + '[\\s\\S]*?\\)', 'ig'),
                    '[' + result
                .map(r => 'require(\'./' + (r.name = niceName(r)) + '\')')
                .join(', ') + `].reduce((acc, r, i) => {
    if(typeof r === 'function') {
        acc[r.name] = r;
    } else if (r && typeof r[Object.keys(r)[0]] === 'function') {
        acc[Object.keys(r)[0]] = r[Object.keys(r)[0]];
        acc[r[Object.keys(r)[0]].name] = r[Object.keys(r)[0]];
    }
    acc[i] = r;
    return acc;
}, {})`.replace(/\n/igm, ''));
            return;
        } else {
            result = result[0] || result;
        }
        newCell = newCell.replace(
            new RegExp('((importer\\.)*import(Notebook)*)\\([^\\)]*?[\'"]'
                       + escapeRegExp(typeof i === 'string' ? i : i[0])
                       + '[\'"][^\\)]*?\\)', 'ig'),
                'require(\'./' + (result.name = niceName(result)) + '\')');
    });
    return addImports(newCell, allImports);
}
module.exports = replaceImports;


export deploy notebook coordinator?

TODO: automatically find missing imports from distrib/Github/3rd party sources in any language?



In [1]:
// assuming you've already run `aws configure`
var fs = require('fs');
var os = require('os');
var path = require('path');
var minimatch = require('minimatch');
var importer = require('../Core');
var {
    execCmd,
    delint,
    getExtension,
    getImportsRecursively,
    replaceImports,
    getPermissions,
    injectImports,
    projectRelatives
} = importer.import([
    'spawn child process',
    'delinting notebooks',
    'cell extension',
    'get imports recursively',
    'replace notebook imports',
    'rpc permissions',
    'inject cells notebooks webpack',
    'fix project paths'
]);

// TODO: exclude Core, defaults to false
function exportAndDeploy(notebook, projectOutput, matchOutput) {
    // these are the top level cells starting the import tree
    let entryCells = importer.interpret(notebook);
    if(!entryCells.length) {
        entryCells = [entryCells];
    }
    if(!notebook || typeof entryCells[0].code === 'undefined') {
        throw new Error('No notebook found!');
    }
    const imports = {};
    for(var k = 0; k < entryCells.length; k++) {
        // TODO: exclude Core
        getImportsRecursively(imports, entryCells[k]);
    }
    const allSearches = Object.keys(imports)
        .reduce((acc, i) => acc.concat([i], ...imports[i]), [])
        .filter((r, i, arr) => arr.indexOf(r) === i);
    const interpretedImports = allSearches
        .reduce((acc, i) => {
            const result = importer.interpret(i);
            if(result.length) {
                acc[i] = result.filter(r => r.questions.length > 0);
            } else if(result.questions.length > 0) {
                acc[i] = result;
            }
            return acc;
        }, {});
    Object.assign(interpretedImports, injectImports(imports, interpretedImports));
    Object.keys(imports).forEach((e, i) => {
        const cell = interpretedImports[e];
        if(cell.questions.length === 0 || cell.questions[0].trim() === '') {
            return;
        }
        let exportedCode = replaceImports(imports[e], cell, interpretedImports);
        if(cell.name === '.js') {
            throw new Error('no filename! ' + JSON.stringify(cell));
        }
        if(cell.name.toLowerCase() === 'readme.js') {
            exportedCode = cell.markdown;
            cell.name = 'readme.md';
        }
        
        // emit the file in every location request
        if(!projectOutput) {
            projectOutput = path.resolve(__dirname);
        }
        if(!matchOutput) {
            matchOutput = {};
        }
        if(typeof matchOutput['**/*'] === 'undefined') {
            matchOutput["**/*"] = path.relative(projectOutput, path.resolve(path.join(__dirname, "../.output"))) + '/';
        }
        Object.keys(matchOutput).forEach(match => {
            if(minimatch(cell.name, match)) {
                // create directory if needed
                const dir = matchOutput[match][matchOutput[match].length-1] === '/'
                    ? matchOutput[match]
                    : (fs.existsSync(matchOutput[match]) && fs.lstatSync(matchOutput[match]).isDirectory()
                       ? matchOutput[match]
                       : path.dirname(matchOutput[match]));
                if(!fs.existsSync(path.join(projectOutput, dir))) {
                    const segs = path.resolve(path.join(projectOutput, dir)).split(/[\/\\]/igm).slice(1);
                    segs.forEach((s, i) => {
                        if(!fs.existsSync('/' + segs.slice(0, i + 1).join('/'))) {
                            fs.mkdirSync('/' + segs.slice(0, i + 1).join('/'));
                        }
                    });
                }
                let filename = path.join(projectOutput, dir, path.basename(matchOutput[match]) === path.basename(dir)
                                         ? cell.name
                                         : path.basename(matchOutput[match]));
                console.log('emitting ' + filename);
                fs.writeFileSync(filename, exportedCode);
            }
        });
    });
    // TODO: output packed cells and cache
    // TODO: zip and upload to AWS
    return Promise.resolve(imports)
        .then(() => projectRelatives(projectOutput))
    //return delint(PROJECT_PATH)
}
module.exports = exportAndDeploy;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
    var project = PROFILE_PATH + '/Documents/subscription.services.eloqua';

    $$.async();
    exportAndDeploy('zuora to eloqua.ipynb', project, {
        "**/*": path.relative(project, path.resolve(path.join(__dirname, "../.output"))) + '/',
        "**/zuora-+(export|renewals|account)*": "./core/",
        "**/!(sync)*eloqua!(-express)*": "./core/",
        "**/aws*": "./upload/",
        "**/notify*": "./notify/",
        "**/+(readme|http-request)*": "./",
    })
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb[0]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[1]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[2]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[3]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[4]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb[1]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\file system.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\U

Not going to work file system.ipynb[]
Not going to work file system.ipynb[]
Not going to work test import.ipynb[]
Not going to work test import.ipynb[]
Not going to work test import.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work demo.ipynb[]
Not going to work math.ipynb[]
Not going to work Angular 2.ipynb[]
Not going to work Angular 2.ipynb[]
Not going to work Angular 2.ipynb[]
Not going to work Angular 2.ipynb[]
Not going to work angular-cli.ipynb[]
Not going to work d3.ipynb[]
Not going to work display angular.ipynb[]
Not going to work display angular.ipynb[]
Not going to work display angular.ipynb[]
Not going to work display angular.ipynb[]
Not going to work gulp.ipynb[]
Not going to work gulp.ipynb[]
Not going to work gulp.ipynb[]
Not going to work gulp.ipynb[]
Not going to work identity s

emitting \Users\brian.cullinan\Documents\jupytangular2\.output\zuora-export-service.js
emitting \Users\brian.cullinan\Documents\subscription.services.eloqua\core\zuora-export-service.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\import-notebook.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\stream-json-using-a-match-function.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\parse-cells-of-types-from-a-notebook.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\run-all-promises-sequentially.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\turn-jupyter-notebooks-in-to-modules.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\find-all-notebooks.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\How-does-node-module-require-work.js
emitting \Users\brian.cullinan\Documents\jupytangular2\.output\read-markdown-leading-up-to-code-cells.js
emitting \Users\brian.cullinan\Document

[]

Search for **/zuora-account-service.js*
Replacing ./zuora-account-service.js -> ../core/zuora-account-service.js in notify\notify-entry-point.js
Search for **/bulk-upload-eloqua.js*
Replacing ./bulk-upload-eloqua.js -> ../core/bulk-upload-eloqua.js in notify\notify-entry-point.js
Replacing ./bulk-upload-eloqua.js -> ../core/bulk-upload-eloqua.js in upload\aws-entry-point.js
Search for **/http-request-pollyfill.js*
Replacing ./http-request-pollyfill.js -> ../http-request-pollyfill.js in core\eloqua-import-service.js
Replacing ./http-request-pollyfill.js -> ../http-request-pollyfill.js in core\eloqua-import-service.spec.js
Replacing ./http-request-pollyfill.js -> ../http-request-pollyfill.js in core\zuora-account-service.js
Replacing ./http-request-pollyfill.js -> ../http-request-pollyfill.js in core\zuora-export-service.js
Replacing ./http-request-pollyfill.js -> ../http-request-pollyfill.js in core\zuora-export-service.spec.js
Search for **/zuora-export-month.js*
Replacing ./zuora-expo

inject cells in to notebooks when built with webpack?


In [ ]:
var importer = require('../Core');
var path = require('path');

// inject cellCache and cellIds loaded in to the interpreter so notebooks don't need to be uploaded, and webpack can tree-shake on functions already packed
// TODO: this should be a test in the Core notebook
function injectImports(imports, interpretedImports) {
    imports['injected imports.ipynb[0]'] = Object.keys(imports)
        .filter(k => k.indexOf('injected imports.ipynb') === -1);
    imports['injected permissions.ipynb[0]'] = [];
    imports['injected cells.ipynb[0]'] = [];
    
    const requires = [];
    const cells = Object.keys(interpretedImports).reduce((acc, k) => {
        const book = typeof interpretedImports[k].length === 'undefined'
            ? [interpretedImports[k]]
            : interpretedImports[k];
        if(typeof acc[book[0].filename] === 'undefined') {
            acc[book[0].filename] = [];
            const notebook = importer.interpret(path.basename(book[0].filename));
            notebook.forEach(cell => {
                if(typeof imports[cell.id] !== 'undefined') {
                    requires.push(JSON.stringify(cell.id) + ': (ctx) => importer.import(' + JSON.stringify(cell.id) + ')');
                }
                cell.markdown.forEach(m => {
                    acc[cell.filename].push({
                        source: m.split(/\n/ig).map(l => l + '\n'),
                        language: cell.language,
                        cell_type: 'markdown'
                    });
                });
                acc[cell.filename].push({
                    source: cell.code.split(/\n/ig).map(l => l + '\n'),
                    language: cell.language,
                    cell_type: 'code'
                });
            });
        }
        return acc;
    }, {});
    const all = getPermissions();
    const permissions = Object.keys(all).reduce((acc, k) => {
        try {
            if(typeof imports[importer.interpret(k).id] !== 'undefined') {
                acc[k] = all[k];
            }
        } catch(e) {
        }
        return acc;
    }, {});
    
    return {
        'injected imports.ipynb[0]': {
            markdown: `
How to inject cells in to webpack?
`.split(/\n/ig).map(c => c + '\n'),
            language: 'javascript',
            cell_type: 'code',
            questions: ['inject cells in to webpack'],
            code: `function getInjected() {
    return {
        ${requires.join(',\n        ')}
    };
}
module.exports = getInjected;
`,
            filename: '../Utilities/notebook.ipynb/injectImports()[0]'
        },
        'injected cells.ipynb[0]': {
            markdown: `
How to inject notebook cache in to webpack?
`.split(/\n/ig).map(c => c + '\n'),
            language: 'javascript',
            cell_type: 'code',
            questions: ['inject notebook cache in to webpack'],
            code: `function getAllCached() {
    return ${JSON.stringify(cells, null, 4)};
}
module.exports = getAllCached;
`,
            filename: '../Utilities/notebook.ipynb/injectImports()[1]'
        },
        'injected permissions.ipynb[0]': {
            markdown: `
How to inject cached permissions in to webpack?
`.split(/\n/ig).map(c => c + '\n'),
            language: 'javascript',
            cell_type: 'code',
            questions: ['inject cached permissions in to webpack'],
            code: `function getCachedPermissions() {
    return ${JSON.stringify(permissions)};
}
module.exports = getCachedPermissions;
`,
            filename: '../Utilities/notebook.ipynb/injectImports()[2]'
        }
    }
}
module.exports = injectImports;
